In [2]:
import imgaug.augmenters as iaa
from skimage import io
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Input klasörü
input_folder = "/content/drive/MyDrive/AITEX/Input"

test_folder = "/content/drive/MyDrive/AITEX/Input/test"
train_folder = "/content/drive/MyDrive/AITEX/Input/train"


train_dataset_df = pd.read_csv("/content/drive/MyDrive/AITEX/train_dataset_df.csv")
test_dataset_df = pd.read_csv("/content/drive/MyDrive/AITEX/test_dataset_df.csv")

In [5]:
train_dataset_df

,index,Defect Image Name,Defect Code,Defect Name
0,80,0071_030_03.png,30,Nep
1,11,0028_019_02.png,19,Fuzzyball
2,10,0029_019_02.png,19,Fuzzyball
3,48,0006_000_02.png,0,No Defect
4,53,0041_019_02.png,19,Fuzzyball
...,...,...,...,...
192,66,0010_000_05.png,0,No Defect
193,66,0085_030_02.png,30,Nep
194,2,0006_002_01.png,2,Broken end
195,13,0027_019_02.png,19,Fuzzyball


In [6]:
test_dataset_df

,index,Defect Image Name,Defect Code,Defect Name
0,52,0044_019_04.png,19,Fuzzyball
1,29,0017_000_06.png,0,No Defect
2,57,0038_019_02.png,19,Fuzzyball
3,17,0019_000_05.png,0,No Defect
4,22,0019_000_03.png,0,No Defect
5,63,0005_000_02.png,0,No Defect
6,79,0072_010_03.png,10,Broken pick
7,88,0005_000_00.png,0,No Defect
8,59,0035_019_02.png,19,Fuzzyball
9,33,0062_022_00.png,22,Cut selvage


In [10]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io

In [12]:
pip install imageio

In [15]:
import imageio

In [16]:
"""
def augment_images(images, num_augmented_images):
    seq = iaa.Sequential([
        iaa.Affine(rotate=(-10, 10)),  # Rastgele rotasyon
        iaa.Fliplr(0.5),  # Rastgele yatay çevirme
        iaa.Flipud(0.5),  # Rastgele dikey çevirme
        iaa.GaussianBlur(sigma=(0, 1.0)),  # Gaussian bulanıklık
        iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Gaussian gürültü
        iaa.Dropout(p=(0, 0.1))  # Rastgele piksel silme
    ], random_order=True)

    augmented_images = []
    for _ in range(num_augmented_images):
        augmented = seq(images=images)
        augmented_images.append(augmented)

    return augmented_images"""

"""
def augment_images(images, num_augmented_images):
    seq = iaa.Sequential([
        iaa.Affine(rotate=(-30, 30)),  # Rastgele rotasyon
        iaa.Fliplr(0.5),  # Rastgele yatay çevirme
        iaa.Flipud(0.5),  # Rastgele dikey çevirme
        iaa.Resize((224, 224), interpolation="linear"),  # Görüntüyü 224 x 224 piksel boyutuna ayarla
        iaa.GaussianBlur(sigma=(0, 0.5)),  # Gaussian bulanıklık
        iaa.AdditiveGaussianNoise(scale=(0, 0.03*255)),  # Gaussian gürültü
    ], random_order=True)

    augmented_images = []
    for _ in range(num_augmented_images):
        augmented = seq(images=images)
        augmented_images.append(augmented)

    return augmented_images
"""


def augment_images(images, num_augmented_images):
    # ImageDataGenerator oluşturalım
    datagen = ImageDataGenerator(
        rotation_range=30,  # Rastgele rotasyon
        horizontal_flip=True,  # Rastgele yatay çevirme
        vertical_flip=True,  # Rastgele dikey çevirme
        zoom_range=0.2,  # Rastgele yakınlaştırma
        width_shift_range=0.2,  # Rastgele yatay kayma
        height_shift_range=0.2,  # Rastgele dikey kayma
        shear_range=0.2,  # Rastgele kesme
        rescale=1./255,  # Görüntü piksellerini normalize etme
        brightness_range=[0.8, 1.2],  # Rastgele parlaklık ayarlama
        fill_mode='nearest',  # Doldurma modu
    )

    augmented_images = []
    for image in images:
        img = imageio.imread(image)  # Görüntü dosyasını okuyalım
        img = np.expand_dims(img, axis=0)
            for _ in range(num_augmented_images):
            augmented = datagen.flow(img, batch_size=1).next()[0]
            augmented_images.append(augmented)

    return augmented_images


num_augmented_images = 100

# Test klasöründeki görüntüleri okuyalım ve augment edelim
test_images = [io.imread(os.path.join(test_folder, img_name)) for img_name in test_dataset_df['Defect Image Name']]
augmented_test_images = augment_images(test_images, num_augmented_images)

# Train klasöründeki görüntüleri okuyalım ve augment edelim
train_images = [io.imread(os.path.join(train_folder, img_name)) for img_name in train_dataset_df['Defect Image Name']]
augmented_train_images = augment_images(train_images, num_augmented_images)


# Test tablosuna augmented verileri ekleyelim
augmented_test_dataset_df = test_dataset_df.copy()
augmented_test_dataset_df['AugmentedImageName'] = [f"augmented_test_{i}.jpg" for i in range(len(augmented_test_images))]
augmented_test_dataset_df['ErrorCode'] = augmented_test_dataset_df['ErrorCode']
augmented_test_dataset_df['ErrorName'] = augmented_test_dataset_df['ErrorName']

# Train tablosuna augmented verileri ekleyelim
augmented_train_dataset_df = train_dataset_df.copy()
augmented_train_dataset_df['AugmentedImageName'] = [f"augmented_train_{i}.jpg" for i in range(len(augmented_train_images))]
augmented_train_dataset_df['ErrorCode'] = augmented_train_dataset_df['ErrorCode']
augmented_train_dataset_df['ErrorName'] = augmented_train_dataset_df['ErrorName']

# Sonuçları gösterelim
print("Test verisi için Augmented Tablo:")
print(augmented_test_dataset_df.head())

print("\nTrain verisi için Augmented Tablo:")
print(augmented_train_dataset_df.head())

<ipython-input-16-87cf2ac067bb>:56: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(image)  # Görüntü dosyasını oku


OSError: Cannot understand given URI: array([[[113, 255],
        [111, 255],
        [114, 255....

In [18]:
# Test klasöründeki görüntüleri okuyalım ve augment edelim
test_images = [io.imread(os.path.join(test_folder, img_name)) for img_name in test_dataset_df['Defect Image Name']]
#augmented_test_images = augment_images(test_images, num_augmented_images)

# Train klasöründeki görüntüleri okuyalım ve augment edelim
train_images = [io.imread(os.path.join(train_folder, img_name)) for img_name in train_dataset_df['Defect Image Name']]
#augmented_train_images = augment_images(train_images, num_augmented_images)

# **MODEL TRAINING**

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from skimage import color

In [21]:
"""X_train_images = augmented_train_images
X_test_images = augmented_test_images"""

X_train_images = train_images
X_test_images = test_images

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_dataset_df["Defect Name"])
y_test = label_encoder.fit_transform(train_dataset_df["Defect Name"])

In [33]:
# Image'leri flat hale getirmeliyiz:
"""X_train_flat = X_train_images.reshape(X_train_images.shape[0], -1)
X_test_flat = X_test_images.reshape(X_test_images.shape[0], -1)"""

"""
X_train_flat = np.array(X_train_images).reshape(X_train_images.shape[0], -1)
X_test_flat = np.array(X_test_images).reshape(X_test_images.shape[0], -1)"""

"""
# Boyut düzenleme
X_train_flat = np.hstack((X_train_images[i].flatten() for i in range(X_train_images.shape[0])))
X_test_flat = np.hstack((X_test_images[i].flatten() for i in range(X_test_images.shape[0])))
"""

"""
X_train_flat = np.vstack([img.flatten() for img in X_train_images])
X_test_flat = np.vstack([img.flatten() for img in X_test_images])"""


# Boyut düzenleme
X_train_flat = np.hstack((X_train_images[i].flatten() for i in range(X_train_images.shape[0])))
X_test_flat = np.hstack((X_test_images[i].flatten() for i in range(X_test_images.shape[0])))

AttributeError: 'list' object has no attribute 'shape'

In [27]:
model = DecisionTreeClassifier(random_state=42)
model.fit(np.array(X_train_images), y_train)

<ipython-input-27-886ec1b3a553>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  model.fit(np.array(X_train_images), y_train)


ValueError: could not broadcast input array from shape (256,4096) into shape (256,)

## **Prediction**

In [ ]:
y_pred = model.predict(X_test_flat)

# **MODEL EVALUATION**

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
# Confusion Matrix:
confusionMatrix = confusion_matrix(y_test, y_pred)
plt.xlabel('Predict')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Accuracy, Precision, Recall ve F1 score değerleri
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label="Defect")
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()